In [1]:
import json,re,os,datetime,time,string
from pathlib import Path
import requests
import langchain
from langchain.llms.base import LLM
from typing import Any, List, Optional,Dict, Mapping, Union,Tuple
from deepeval.models.base_model import DeepEvalBaseLLM
import collections
import traceback,random
import pandas as pd
import numpy as np
import evalplus.sanitize,evalplus.syncheck

d:\Anaconda\envs\vllm\lib\site-packages\deepeval\__init__.py:41: UserWarning: You are using deepeval version 0.21.27, however version 0.21.36 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [64]:
INFINI_API = "sk-c7cssl4bkglsrwf2"
INFINI_API_2 = "sk-c7erk6qaqhkz5t72"
INFINI_API_List = ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"]

In [65]:
modelList = {
    'llama-3-70b-instruct':"Llama3系列是由Meta开发的Llama系列全新的第三代版本，包含一系列预训练和指令调优的文本生成式模型。Llama3基于优化后的Transformer架构，预训练过程中使用了超过15T tokens的数据，调优后的模型使用SFT和RLHF，以更好地贴合人类对可用性和安全性的偏好。Llama3-70b-Instruct是此系列里，700亿参数的指令调优的模型，针对对话场景用例进行了优化，并在常见的行业基准测试中超越了许多可用的开源聊天模型。Llama3-70b-Instruct支持模型上下文至8k tokens，该模型的数据的知识截止日期为2023年12月。",
    'llama-3-8b-instruct':"Llama3系列是由Meta开发的Llama系列全新的第三代版本，包含一系列预训练和指令调优的文本生成式模型。Llama3基于优化后的Transformer架构，预训练过程中使用了超过15T tokens的数据，调优后的模型使用SFT和RLHF，以更好地贴合人类对可用性和安全性的偏好。Llama3-8b-Instruct是此系列里，80亿参数的指令调优的模型，针对对话场景用例进行了优化，并在常见的行业基准测试中超越了许多可用的开源聊天模型。Llama3-8b-Instruct支持模型上下文至8k tokens，该模型的数据的知识截止日期为2023年3月。",
    'chatglm3':"ChatGLM3是智谱AI与清华KEG实验室发布的闭源模型，支持 8K 上下文，经过海量中英标识符的预训练与人类偏好对齐训练，相比一代模型在 MMLU、C-Eval、GSM8K 分别取得了16%、36%、280%的提升，并登顶中文任务榜单C-Eval。适用于对知识量、推理能力、创造力要求较高的场景，比如广告文案、小说写作、知识类写作、代码生成等。",
    'chatglm2-6b':"ChatGLM2-6b 是由智谱开发的 ChatGLM 系列的第二代版本，支持中英双语的60亿参数规模的开源模型。在保留了初代模型对话流畅、部署门槛较低等众多优秀特性的基础之上，在 MMLU、C-Eval、GSM8K、BBH等主流学术数据集上，都得到了显著的性能提升，并通过基于 FlashAttention 技术，将对话模型的上下文长度（Context Length）提升至 8k tokens，允许更多轮次的对话。",
    'chatglm2-6b-32k':"ChatGLM2-6b 是由智谱开发的 ChatGLM 系列的第二代版本，支持中英双语的60亿参数规模的开源模型。相较于ChatGLM2-6B，ChatGLM2-6b-32k支持更长的模型上下文至32k tokens。",
    'infini-megrez-7b':"由无问芯穹公司自主研发的70亿参数大语言模型。在逻辑推理、对话能力等方面有优秀的性能表现。配合无问芯穹自研高效推理引擎，同时支持Nvidia和AMD的GPU，具备更快的推理速度，在性能表现方面更上一层楼。",
    'llama-2-7b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-7b-chat是其中70亿的主流参数大小的模型，适用于chat场景，更擅长英文相关的内容。模型支持 4k tokens上下文。",
    'llama-2-13b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-7b-chat是其中70亿的主流参数大小的模型，适用于chat场景，更擅长英文相关的内容。模型支持 4k tokens上下文。",
    'llama-2-70b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-70b-chat是其中700亿参数的大模型，适用于chat场景，更擅长英文相关的内容，相较该系列里其他规模的的模型，有更强的综合能力。模型支持 4k tokens上下文。",
    'llama-2-70b':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-70b-base是其中700亿参数的基础大模型，适用于通用语言任务场景，更擅长英文相关的内容，相较该系列里其他规模的的模型，有更强的综合能力。模型支持 4k tokens上下文。",
    'baichuan2-7b-chat':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-7b-chat是130亿参数规模用于对话的模型，在C-Eval、MMLU、CMMLU等主流评测数据集上都有不俗的表现。该基模型支持4k tokens上下文。",
    'baichuan2-13b-chat':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-13b-chat是130亿参数规模用于对话的模型，在C-Eval、MMLU、CMMLU等主流评测数据集上都有不俗的表现。该基模型支持8k tokens上下文。",
    'baichuan2-13b-base':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-13b-base是130亿参数规模的基础模型，适用于通用对话和文本续写，较chat模型更适合于复杂场景的微调后使用。该基模型支持4k tokens上下文。",
    'chatglm3-6b':"ChatGLM3-6b 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源模型。ChatGLM3采用了全新设计的 Prompt 格式，并原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。模型支持 8k tokens上下文。",
    'chatglm3-6b-32k':"ChatGLM3-6b 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源模型。相较于ChatGLM之前系列的模型，ChatGLM3采用了更多样的训练数据，并原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。ChatGLM3-6b-32k在ChatGLM3-6b 基础上进一步强化了对于长文本的理解能力，能够更好的处理最多32k tokens长度的上下文。",
    'chatglm3-6b-base':"ChatGLM3-6b-base 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源的基础模型。ChatGLM3-6B-Base 采用了更多样的训练数据、更充分的训练步数和更合理的训练策略。基础模型更适合于复杂场景的微调后使用，该基模型支持32k tokens上下文。",
    'qwen-7b-chat':"通义千问-7B-chat（Qwen-7B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的70亿参数规模的大语言模型。相较于Qwen-7B-Base模型，Qwen-7B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 8k tokens上下文。",
    'qwen-14b-chat':"通义千问-14B-chat（Qwen-14B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的140亿参数规模的大语言模型。相较于Qwen-14B-Base模型，Qwen-14B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 8k tokens上下文。",
    'qwen-72b-chat':"通义千问-72B-chat（Qwen-72B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的720亿参数规模的大语言模型。相较于Qwen-72B-Base模型，Qwen-72B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 32k tokens上下文。",
    'qwen-72b':"通义千问-72B（Qwen-72B）是阿里云研发的通义千问大模型系列的720亿参数规模的模型。Qwen-72B是基于Transformer的大语言模型, 在超大规模的预训练数据上进行训练得到。预训练数据类型多样，覆盖广泛，包括大量网络文本、专业书籍、代码等。模型支持 32k tokens上下文。",
    'qwen1.5-7b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-7b-chat是其中专用于chat场景的70亿参数的主流大小模型。",
    'qwen1.5-14b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-14b-chat是其中专用于chat场景的140亿参数的主流大小模型。",
    'qwen1.5-72b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-72b-chat是其中专用于chat场景的720亿参数的大模型。",
    'qwen1.5-72b':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-72b-base是其中的720亿参数的基础大模型，适合多种场景的使用。",
}
answerModelList = [    
    'llama-3-8b-instruct',
    # 'chatglm3',
    # 'chatglm2-6b',
    # 'chatglm2-6b-32k',
    'infini-megrez-7b',
    'llama-2-7b-chat',
    'llama-2-13b-chat',
    'llama-2-70b-chat',
    'llama-2-70b',
    'baichuan2-7b-chat',
    'baichuan2-13b-chat',
    'baichuan2-13b-base',
    # 'chatglm3-6b',
    # 'chatglm3-6b-32k',
    # 'chatglm3-6b-base',
    'qwen-7b-chat',
    'qwen-14b-chat',
    'qwen-72b-chat',
    'qwen-72b',
    'qwen1.5-7b-chat',
    'qwen1.5-14b-chat',
    'qwen1.5-72b',
]

In [66]:
def LLMCompletions(prompt,modelName:str = "infini-megrez-7b",INFINI_API = "sk-c7cssl4bkglsrwf2",returnContent:bool = True,**kwargs):
    url = "https://cloud.infini-ai.com/maas/"+modelName+"/nvidia/chat/completions"
    payload = {
        "model": "string",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
        "temperature": kwargs["temperature"] if "temperature" in kwargs else 0.7,
        "top_p": kwargs["top_p"] if 'top_p' in kwargs else 1,
        "top_k": kwargs['top_k'] if 'top_k' in kwargs else -1,
        "n": kwargs['n'] if 'n' in kwargs else 1,
        "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else None,
        "stop": kwargs['stop'] if 'stop' in kwargs else None,
        "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else 0,
        "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else 0
    }
    headers = {
            'Content-Type': "application/json",
            'Accept': "*/*",
            'Authorization': "Bearer "+INFINI_API,
    } 
    response = requests.post(url, json=payload, headers=headers,)
    if response.status_code == 200:
        response.encoding = 'utf-8'
        data = response.json()
        content = data['choices'][0]['message']['content']
        
        content = content.replace(',\n}','\n}')
        if returnContent:
            return content
        try:
            content = json.loads(content)
        except:
            content = content.replace('\n','')
        data['choices'][0]['message']['content'] = content
        
        return json.dumps(data['choices'][0]['message']['content'])

    else:
        print(response.status_code)
        return "Cannot connect to the model "+modelName

In [67]:
data_format_ins = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'retrieval_context':None,
    'time':-1
}
def joinErrorToData(errorFile,saveFile):
    with open(errorFile, 'r') as ef:
        data_ef = json.load(ef)
    fileName = data_ef['fileName']
    with open(saveFile) as sf:
        data_sf = json.load(sf)
    if fileName != data_sf['fileName']:
        print('FileName not match')
        return 
    if not data_ef['data']:
        print('The Errors of this ErrorFile all have been solved!')
        return 
    errorItem = []
    for item in data_ef['data']:
        data_format_ins['id'] = item['id']
        model = item['AnswerModel']
        data_format_ins['AnswerModel'] = model
        prompt = item['input']
        data_format_ins['input'] = prompt
        data_format_ins['expected_output'] = item['expected_output']
        start = time.perf_counter_ns()
        actual_output =  LLMCompletions(prompt,modelName=model,INFINI_API=INFINI_API_2)
        end = time.perf_counter_ns()
        delta = end-start
        idx = 0
        while actual_output == "Cannot connect to the model "+model and idx<2:
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model,INFINI_API=INFINI_API_2)
            end = time.perf_counter_ns()
            delta = end-start
            idx += 1
        if actual_output == "Cannot connect to the model "+model:
            errorItem.append(item)
            print("[error]:\t"+str(errorItem[-1]))
            continue
        print(actual_output)
        data_format_ins['time'] = delta
        data_format_ins['actual_output'] = actual_output
        data_sf['data'].append(data_format_ins.copy())
    data_ef['data'] = errorItem
    with open(saveFile,'w') as saveF:
        json.dump(data_sf,saveF)
    with open(errorFile,'w') as error:
        json.dump(data_ef,error)
    if errorItem:
        print("There are still some errors! ")
    else:
        print('The Errors of this ErrorFile all have been solved!')

In [68]:
def removeDeprecatedModel(filePath):
    with open(filePath) as f:
        data = json.load(f)
    new_data = []
    for item in data['data']:
        if item['AnswerModel']  in answerModelList:
            new_data.append(item)
    data['data'] = new_data[:]
    with open(filePath,'w') as f:
        json.dump(data,f)

In [69]:
# removeDeprecatedModel('./data/1-2_1_low_freq_ent_sample.json')

In [70]:
# joinErrorToData('./data/4-2_r_with_triples_sampleError.json','./data/4-2_r_with_triples_sample.json')

In [71]:
# diff = collections.Counter()

In [72]:
# qapairs = []
# model_input_id = []
# with open('./data/1-1_2_high_freq_ent_sample.json') as saveF:
#     data = json.load(saveF)
# with open('E:/Repository/KoLA/Sample_Data/1-1_2_high_freq_ent_sample.json') as file:
#     data_o = json.load(file)
# instructions = data_o['adapter_spec']['instructions']
# for item in data_o['request_states']:
#     qapairs.append((item['instance']['id'],instructions+'\n'+item['instance']['input']['text']))
# for item in data['data']:
#     model_input_id.append([item['AnswerModel'],item['input'],item['id']])
# check = [ False for i in range(len(model_input_id))]
# for model in answerModelList:
#     for id,text in qapairs:
#         for i,item in enumerate(model_input_id):
#             if model == item[0] and text == item[1] and id == item[2]:
#                 check[i] = True

In [73]:
# for model,text,id in model_input_id:
#     diff[model] += 1
#     diff[text] += 1
#     diff[id] += 1

In [74]:
### deprecated ChatLLM,CustomLLM
# class ChatLLM(LLM):
#     modelName = "qwen1.5-72b-chat"
#     INFINI_API = "sk-c7cssl4bkglsrwf2"
#     temperature = 0.8
#     top_p = 1
#     top_k = -1
#     n = 1
#     max_tokens:int = None
#     stop:Optional[List[str]] = None
#     presence_penalty = 0
#     frequency_penalty = 0
    
#     headers = {
#             'Content-Type': "application/json",
#             'Accept': "*/*",
#             'Authorization': "Bearer "+INFINI_API,
#     }   
#     @property
#     def _llm_type(self)->str:
#         return "ChatLLM"
#     @property
#     def _identifying_params(self)->Mapping[str,Any]:
#         _param_dict = {
#             "modelName":ChatLLM().modelName,
#             "INFINI_API":ChatLLM().INFINI_API,
#             "stream":bool(ChatLLM().stream),
#             "temperature":ChatLLM().temperature,
#             "top_p":ChatLLM().top_p,
#             "top_k":ChatLLM().top_k,
#             "n":ChatLLM().n,
#             "max_tokens":ChatLLM().max_tokens,
#             "stop":ChatLLM().stop,
#             "presence_penalty":ChatLLM().presence_penalty,
#             "frequency_penalty":ChatLLM().frequency_penalty,
#         }
#         return _param_dict
#     @classmethod  
#     def _call(self, prompt: str, stop: Optional[List[str]]= None,  **kwargs: Any) -> str:
#         url = "https://cloud.infini-ai.com/maas/"+ChatLLM().modelName+"/nvidia/chat/completions"
#         payload = {
#             "model": "string",
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": prompt
#                 }
#             ],
#             "stream": False,
#             "temperature": kwargs["temperature"] if "temperature" in kwargs else ChatLLM().temperature,
#             "top_p": kwargs["top_p"] if 'top_p' in kwargs else ChatLLM().top_p,
#             "top_k": kwargs['top_k'] if 'top_k' in kwargs else ChatLLM().top_k,
#             "n": kwargs['n'] if 'n' in kwargs else ChatLLM().n,
#             "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else ChatLLM().max_tokens,
#             "stop": stop if stop else ChatLLM().stop,
#             "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else ChatLLM().presence_penalty,
#             "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else ChatLLM().frequency_penalty
#         }
#         response = requests.post(url, json=payload, headers=ChatLLM().headers)
#         if response.status_code == 200:
#             response.encoding = 'utf-8'
#             data = response.json()
#             content = data['choices'][0]['message']['content']
#             if isinstance(content,str):    
#                 content = content.replace(',\n}','\n}')
#                 content = content.replace(']\n}',']}')
#                 content = content.replace('\\','\\\\')
#                 flag = False
#                 if 'statements' in content:
#                     regex = re.compile('\"statements\":\s+\[.*\]\}',re.DOTALL)
#                     flag = True
#                 elif 'verdicts' in content:
#                     regex = re.compile('\"verdicts\":\s+\[.*\]\}', re.DOTALL) 
#                     flag = True
#                 if flag:
#                     matchStr =regex.search(content)
#                     if matchStr:
#                         content = '{'+matchStr.group()
#             try:
#                 content = json.loads(content)
#             except:
#                 pass
#             data['choices'][0]['message']['content'] = content
                
#             return json.dumps(data['choices'][0]['message']['content'])

#         else:
#             return "Cannot connect to the model "+ChatLLM().modelName
#     def setParameter(self,**kwargs):
#         self.temperature = kwargs["temperature"] if "temperature" in kwargs else ChatLLM().temperature
#         self.top_p = kwargs['top_p'] if 'top_p' in  kwargs else ChatLLM().top_p
#         self.top_k = kwargs['top_k'] if 'top_k' in  kwargs else ChatLLM().top_k
#         self.n = kwargs['n'] if 'n' in kwargs else ChatLLM().n
#         self.max_tokens = kwargs['max_tokens'] if 'max_tokens' in kwargs else ChatLLM().max_tokens
#         self.stop = kwargs['stop'] if 'stop' in kwargs else ChatLLM().stop
#         self.presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else ChatLLM().presence_penalty
#         self.frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else ChatLLM().frequency_penalty

# class CustomLLM(DeepEvalBaseLLM):
#     def __init__(
#         self,
#         model
#     ):
#         self.model = model

#     def load_model(self):
#         return self.model

#     def generate(self, prompt: str) -> str:
#         chat_model = self.load_model()
#         return chat_model.invoke(prompt)
#     async def a_generate(self, prompt: str) -> str:
#         chat_model = self.load_model()
#         res = await chat_model.ainvoke(prompt)
#         return res

#     def get_model_name(self):
#         return "CustomLLM"

# custom_model = ChatLLM()

# evaluateModel = CustomLLM(model=custom_model)

In [75]:
path = []

In [76]:
class ChatLLM(LLM):
    @property
    def modelName(self)->str:
        return "qwen1.5-72b-chat"
    @property
    def INFINI_API_List(self)->List[str]:
        return ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"]
    @property
    def temperature(self)->float:
        return 0.7
    @property
    def top_p(self)->float:
        return 0.1
    @property
    def top_k(self)->int:
        return -1
    @property
    def n(self)->int:
        return 1
    @property
    def max_tokens(self)->int:
        return None
    @property
    def stop(self)->Optional[List[str]]:
        return None
    @property
    def presence_penalty(self)->float:
        return 0
    @property
    def frequency_penalty(self)->float:
        return 0
    def getHeader(self,index_api):  
        headers = {
            'Content-Type': "application/json",
            'Accept': "*/*",
            'Authorization': "Bearer "+self.INFINI_API_List[index_api%len(self.INFINI_API_List)],
        }
        return headers
    @property
    def _llm_type(self)->str:
        return "ChatLLM"
    @property
    def _identifying_params(self)->Mapping[str,Any]:
        _param_dict = {
            "modelName":self.modelName,
            "INFINI_API":self.getHeader(self.__fields__['index_api'] if 'index_api' in self.__fields__ else 0),
            "stream":bool(self.stream),
            "temperature":self.temperature,
            "top_p":self.top_p,
            "top_k":self.top_k,
            "n":self.n,
            "max_tokens":self.max_tokens,
            "stop":self.stop,
            "presence_penalty":self.presence_penalty,
            "frequency_penalty":self.frequency_penalty,
        }
        return _param_dict

    def _call(self, prompt: str, stop: Optional[List[str]]= None, run_manager= None,**kwargs: Any) -> str:
        url = "https://cloud.infini-ai.com/maas/"+str(self.modelName)+"/nvidia/chat/completions"
        payload = {
            "model": "string",
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False,
            "temperature": kwargs["temperature"] if "temperature" in kwargs else self.temperature,
            "top_p": kwargs["top_p"] if 'top_p' in kwargs else self.top_p,
            "top_k": kwargs['top_k'] if 'top_k' in kwargs else self.top_k,
            "n": kwargs['n'] if 'n' in kwargs else self.n,
            "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens,
            "stop": stop if stop else self.stop,
            "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty,
            "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
        }
        index = 0
        if 'index_api' not in self.__fields__:
            self.__fields__['index_api'] = -1
        index_api = self.__fields__['index_api']+1
        length = len(self.INFINI_API_List)
        while index < length:
            response = requests.post(url, json=payload, headers=self.getHeader(index_api))
            if response.status_code == 200:
                response.encoding = 'utf-8'
                data = response.json()
                print("response json success")
                content = data['choices'][0]['message']['content']
                if isinstance(content,str):   
                    content = content.replace(',\n}','\n}')
                    content = content.replace(']\n}',']}')
                    if 'statements' in content:
                        regex = re.compile('\"statements\":\s+\[.*\]\}',re.DOTALL)
                        matchStr =regex.search(content)
                        if matchStr:
                            content = '{'+matchStr.group()
                    elif 'verdicts' in content:
                        regex = re.compile('\"verdicts\":\s+\[.*\]\}',re.DOTALL)
                        matchStr =regex.search(content)
                        if matchStr is not None:
                            content ='{' +matchStr.group()
                            regex = re.compile("\"reason\":(.*?)\}",re.DOTALL)
                            matchStr = regex.findall(content)
                            for string in matchStr:
                                tmp = string.strip()[1:-1].replace('"','\\\"')
                                tmp = '\"'+tmp+"\""
                                content = content.replace(string,tmp)
                if isinstance(content,str):
                    return content
                data['choices'][0]['message']['content'] = content
                return json.dumps(data['choices'][0]['message']['content'])

            index += 1
            index_api =  (index_api+1)%length
            self.__fields__['index_api'] = index_api
            print(response.status_code)
            try:
                print(response.json())
            except:
                pass
            time.sleep(1)
        return "Cannot connect to the model "+self.modelName
    def setParameter(self,**kwargs):
        self.temperature = kwargs["temperature"] if "temperature" in kwargs else self.temperature
        self.top_p = kwargs['top_p'] if 'top_p' in  kwargs else self.top_p
        self.top_k = kwargs['top_k'] if 'top_k' in  kwargs else self.top_k
        self.n = kwargs['n'] if 'n' in kwargs else self.n
        self.max_tokens = kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens
        self.stop = kwargs['stop'] if 'stop' in kwargs else self.stop
        self.presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty
        self.frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
    

class CustomLLM(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model
    def load_model(self):
        return self.model
    def generate(self, prompt: str) -> str:
        global path
        chat_model = self.load_model()
        ret = chat_model.invoke(prompt)
        idx = 0
        if ret == "Cannot connect to the model "+self.get_model_name() and idx<2:
            time.sleep(5)
            ret = chat_model.invoke(prompt)
            idx += 1
        return ret
    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        try:
            return self.model.modelName
        except:
            return "CustomLLM"
custom_model = ChatLLM()
evaluateModel = CustomLLM(model=custom_model)

In [77]:
content = "\"verdicts\": [{\"verdict\":\"yes\"} , {\"verdict\":\"no\",\"reason\":\"123\"}]"
regex = re.compile("\{\s*\"verdict\"\s*:.*?\}")
matchStr = regex.findall(content)
matchStr

['{"verdict":"yes"}', '{"verdict":"no","reason":"123"}']

In [78]:
metrics_format_example = {
    'metric_metadata':{
        'metric':None,
        'threshold':0,
        'success':True,
        'score':0.8,
        'reason':'',
        'strictMode': False,
        'evaluationModel': 'CustomLLM',
        'evaluationCost': 0
    },
    'metric_configuration': {
        'threshold': 0.5,
        'evaluation_model': 'CustomLLM',
        'strict_mode': False,
        'include_reason': True
    }
}

data_format_example = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'retrieval_context':None,
    'cached_metrics_data':[
        {
            'metric_metadata':{
                'metric':None,
                'success':True,
                'score':0.8,
                'reason':'',
                'statements':'',
                'verdicts':'',
                'evaluationCost': 0
            },
            'metric_configuration': {
                'threshold': 0.5,
                'evaluation_model': 'CustomLLM',
                'strict_mode': False,
                'include_reason': True
            }
        },
        metrics_format_example
    ]
}

        

In [79]:
for dirName,subDirName ,fileNames in os.walk('./data'):
    print(fileNames)

[]
['data.json', 'data.jsonl', 'infini-megrez-7b_samples.jsonl']
['baichuan2-13b-base.jsonl', 'baichuan2-13b-chat.jsonl', 'baichuan2-7b-chat.jsonl']
['baichuan2-13b-base.jsonl', 'baichuan2-13b-chat.jsonl', 'baichuan2-7b-chat.jsonl', 'infini-megrez-7b.jsonl', 'llama-2-13b-chat.jsonl', 'llama-2-70b-chat.jsonl', 'llama-2-70b.jsonl', 'llama-2-7b-chat.jsonl', 'qwen-14b-chat.jsonl', 'qwen-72b-chat.jsonl', 'qwen-72b.jsonl', 'qwen-7b-chat.jsonl', 'qwen1.5-14b-chat.jsonl', 'qwen1.5-72b.jsonl', 'qwen1.5-7b-chat.jsonl']
['baichuan2-13b-base.jsonl', 'baichuan2-13b-chat.jsonl', 'baichuan2-7b-chat.jsonl', 'infini-megrez-7b.jsonl', 'infini-megrez-7b_samples.jsonl', 'llama-2-13b-chat.jsonl', 'llama-2-70b-chat.jsonl', 'llama-2-70b.jsonl', 'llama-2-7b-chat.jsonl', 'qwen-14b-chat.jsonl', 'qwen-72b-chat.jsonl', 'qwen-72b.jsonl', 'qwen-7b-chat.jsonl', 'qwen1.5-14b-chat.jsonl', 'qwen1.5-72b.jsonl', 'qwen1.5-7b-chat.jsonl']
[]
['1-1_2_high_freq_ent_sample - copy.json', '1-1_2_high_freq_ent_sample.json', '1-2

In [80]:
fileNames = ['1-3_r_1_simple_sample_sample.json', '2-1_COPEN++csj_sample.json', '2-2_COPEN++cpj_sample.json', '2-3_COPEN++cic_sample.json', '2-4_FewNERD++inter_sample.json', '2-4_FewNERD++intra_sample.json', '2-4_FewNERD++supervised_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json', '2-7_MAVEN-ERE_sample.json', '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json', '4-1_without_triples_sample.json', '4-1_with_triples_sample.json', '4-2_r_without_triples_sample.json', '4-2_r_with_triples_sample.json']

In [81]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric,FaithfulnessMetric,HallucinationMetric,BaseMetric
from deepeval.test_case import LLMTestCase

In [82]:
metrics  = [
    AnswerRelevancyMetric(
        threshold=0.5,
        model=evaluateModel,
        include_reason=True
    ),
    HallucinationMetric(
        threshold=0.5,
        model=evaluateModel,
        include_reason=True
    )
]


In [83]:
# with open('./eval/error/2-1_COPEN++csj_sample.json') as f:
#     data = json.load(f)
# data['data'][0]

In [84]:
# test_case = LLMTestCase(input=data['data'][0]['input'],
#                         actual_output=data['data'][0]['actual_output'],
#                         context=[data['data'][0]['expected_output']])

In [85]:
# metrics[0].measure(test_case)

In [86]:
def evaluate_file(filename:Union[str,Path],save_file:Union[str,Path],error_file:Union[str,Path],force_save:bool = False,metrics:List[BaseMetric] = [AnswerRelevancyMetric(threshold=0.5,model=evaluateModel,include_reason=True),HallucinationMetric(threshold=0.5,model=evaluateModel,include_reason=True)]):
    """_summary_
        the function is used to evaluate the LLM output saved in `filename` by the metric in `metrics`,the successful eval results will be saved into `save_file` and the error item will be saved into `error_file`
    
    Args:
        `filename` (Union[str,Path]): the filename saves the LLM generation results
        `save_file` (Union[str,Path]): the filename will save the evaluate results
        `error_file` (Union[str,Path]): the filename will save the error eval item
        `force_save` (bool, optional): if the value is `True`,function will rerun all eval item in `filename` and directly override the `save_file` and `error_file`. 
                    Defaults to False.
        `metrics` (List[BaseMetric], optional): a list of evaluation metrics. 
                    Defaults to [AnswerRelevancyMetric(threshold=0.5,model=evaluateModel,include_reason=True),HallucinationMetric(threshold=0.5,model=evaluateModel,include_reason=True)].
    """
    def is_same_eval_item(x,item):
        if x['id'] == item['id'] and x['input'] == item['input'] and x['actual_output'] == item['actual_output'] and x['AnswerModel'] == item['AnswerModel'] and x['expected_output'] == item['expected_output']:
            return True
        return False
    with open(filename,'r') as f:
        data = json.load(f)
    save,error = dict(),dict()
    if  Path(save_file).is_file() and not force_save:
        with open(save_file) as f:
            save = json.load(f)
        if 'fileName' in save and  save['fileName'] != data['fileName']:
            print("The save_file does not match the file name!")
            return
    else:
        save = {'fileName':data['fileName'],'class':data['class'],'data':[]}

    if Path(error_file).is_file() and not force_save:
        with open(error_file) as f:
            error = json.load(f)
        if 'fileName' in error and  error['fileName'] != data['fileName']:
            print("The error_file does not match the file name!")
            return
    else:
        error = {'fileName' :data['fileName'],'class':data['class'],'data':[]}
    for metric in metrics:
        for item in data['data']:
            data_format_ins = {
                'id':0,
                'AnswerModel':'',
                'input':'',
                'actual_output':'',
                'expected_output':None,
                'retrieval_context':None,
                'cached_metrics_data':[]
            }
            metrics_format_ins = {
                'metric_metadata':{
                    'metric':None,
                    'success':True,
                    'score':0.8,
                    'reason':'',
                    'statements':'',
                    'verdicts':'',
                    'evaluationCost': 0
                },
                'metric_configuration': {
                    'threshold': 0.5,
                    'evaluation_model': 'CustomLLM',
                    'strict_mode': False,
                    'include_reason': True
                }
            }
            errors_format_ins = {
                'id':0,
                'AnswerModel':'',
                'input':'',
                'actual_output':'',
                'expected_output':None,
                'retrieval_context':None,
                'cached_metrics_data':[
                    {            
                        'metric_metadata':{
                            'metric':None,
                        },
                        'metric_configuration': {
                            'threshold': 0.5,
                            'evaluation_model': 'CustomLLM',
                            'strict_mode': False,
                            'include_reason': True
                        }
                    }
                ]
            }
            
            data_format_ins['id'] = item['id']
            data_format_ins['AnswerModel'] = item['AnswerModel']
            data_format_ins['input'] = item['input']
            data_format_ins['actual_output'] = item['actual_output']
            data_format_ins['expected_output'] = item['expected_output']
            tag = False
            for x in save['data']:
                if is_same_eval_item(item,x):
                    for metric_data in x['cached_metrics_data']:
                        if metric_data['metric_metadata']['metric'] == metric.__name__:
                            if metric_data['metric_configuration']['threshold'] == metric.threshold and metric_data['metric_configuration']['evaluation_model'] == metric.evaluation_model and metric_data['metric_configuration']['strict_mode'] == metric.strict_mode and metric_data['metric_configuration']['include_reason'] == metric.include_reason:
                                tag = True
                                print("HAVE:")
                                print(x)
                                break
            if tag:
                continue
            test_case = LLMTestCase(
                input= item['input'],
                actual_output=item['actual_output'],
                context=[item['expected_output']],
            )
            try:
                metric.measure(test_case)
            except Exception as e:
                print(e)
                path.append(e)
                errors_format_ins['id'] = item['id']
                errors_format_ins['AnswerModel'] = item['AnswerModel']
                errors_format_ins['input'] = item['input']
                errors_format_ins['actual_output'] = item['actual_output']
                errors_format_ins['expected_output'] = item['expected_output']
                errors_format_ins['cached_metrics_data'][0]['metric_metadata']['metric'] = metric.__name__
                errors_format_ins['cached_metrics_data'][0]['metric_configuration'] = {'threshold':metric.threshold,'evaluation_model':metric.evaluation_model,'strict_mode':metric.strict_mode,'include_reason':metric.include_reason}
                error['data'].append(errors_format_ins.copy())
                print(errors_format_ins)
                with open(error_file,'w') as f:
                    json.dump(error,f,indent=4)
                continue
            metrics_format_ins['metric_metadata']['metric'] = metric.__name__
            metrics_format_ins['metric_metadata']['score'] = metric.score
            metrics_format_ins['metric_metadata']['success'] = metric.is_successful()
            metrics_format_ins['metric_metadata']['reason'] = metric.reason
            
            metrics_format_ins['metric_metadata']['statements'] = getattr(metric,'statements','')
            metrics_format_ins['metric_metadata']['verdicts'] = str(getattr(metric,'verdicts',''))
            metrics_format_ins['metric_metadata']['evaluationCost'] = metric.evaluation_cost

            metrics_format_ins['metric_configuration']['threshold'] = metric.threshold
            metrics_format_ins['metric_configuration']['strict_mode'] = metric.strict_mode
            metrics_format_ins['metric_configuration']['evaluation_model'] = metric.evaluation_model
            metrics_format_ins['metric_configuration']['include_reason'] = metric.include_reason

            data_format_ins['cached_metrics_data'].append(metrics_format_ins.copy())
            flag = True
            for each in save['data']:
                if is_same_eval_item(each, data_format_ins):
                    each['cached_metrics_data'].append(metrics_format_ins.copy())
                    flag = False
                    break
            if flag:
                save['data'].append(data_format_ins.copy())
                print(data_format_ins)
            with open(save_file,'w') as f:
                json.dump(save,f,indent=4)

In [87]:
def joinEvalErrorToData(errorFile:Union[str,Path],saveFile:Union[str,Path])->None:
    """_summary_
        the function is used to rerun the error item in the `errorFile` and append the results into the `saveFile` 
        
    Args:
        `errorFile` (Union[str,Path]): the JSON file saves the error item in the before running
        `saveFile` (Union[str,Path]): the JSON file saves the pass result
        
    Returns:
        None: the result will override the original file 
    """
    def is_same_eval_item(x,item):
        if x['id'] == item['id'] and x['input'] == item['input'] and x['actual_output'] == item['actual_output'] and x['AnswerModel'] == item['AnswerModel'] and x['expected_output'] == item['expected_output']:
            return True
        return False
    with open(errorFile,'r') as f:
        data_er = json.load(f)
    with open(saveFile,'r') as f:
        data_sv = json.load(f)
    if data_er['fileName'] != data_sv['fileName']:
        print("The save_file does not match the error_file!")
        return
    error = {'fileName' :data_er['fileName'],'class':data_er['class'],'data':[]}
    while data_er['data']:
        item = data_er['data'].pop()
        data_format_ins = {
            'id':0,
            'AnswerModel':'',
            'input':'',
            'actual_output':'',
            'expected_output':None,
            'context':None,
            'retrieval_context':None,
            'cached_metrics_data':[]
        }
        metrics_format_ins = {
            'metric_metadata':{
                'metric':None,
                'success':True,
                'score':0.8,
                'reason':'',
                'statements':'',
                'verdicts':'',
                'evaluationCost': 0
            },
            'metric_configuration': {
                'threshold': 0.5,
                'evaluation_model': 'CustomLLM',
                'strict_mode': False,
                'include_reason': True
            }
        }
        errors_format_ins = {
            'id':0,
            'AnswerModel':'',
            'input':'',
            'actual_output':'',
            'expected_output':None,
            'retrieval_context':None,
            'cached_metrics_data':[
                {            
                    'metric_metadata':{
                        'metric':None,
                    },
                    'metric_configuration': {
                        'threshold': 0.5,
                        'evaluation_model': 'CustomLLM',
                        'strict_mode': False,
                        'include_reason': True
                    }
                }
            ]
        }
        data_format_ins['id'] = item['id']
        data_format_ins['AnswerModel'] = item['AnswerModel']
        data_format_ins['input'] = item['input']
        data_format_ins['actual_output'] = item['actual_output']
        data_format_ins['expected_output'] = item['expected_output']
        
        tag = False
        for x in data_sv['data']:
            if is_same_eval_item(item,x):
                for metric_data in x['cached_metrics_data']:
                    if metric_data['metric_metadata']['metric'] == item['cached_metrics_data'][0]['metric_metadata']['metric']:
                        if metric_data['metric_configuration']['threshold'] == item['cached_metrics_data'][0]['metric_configuration']['threshold'] and metric_data['metric_configuration']['evaluation_model'] == item['cached_metrics_data'][0]['metric_configuration']['evaluation_model'] and metric_data['metric_configuration']['strict_mode'] == item['cached_metrics_data'][0]['metric_configuration']['strict_mode'] and metric_data['metric_configuration']['include_reason'] == item['cached_metrics_data'][0]['metric_configuration']['include_reason']:
                            tag = True
                            print("HAVE:")
                            print(x)
                            break
        if tag:
            continue
        test_case = LLMTestCase(
            input= item['input'],
            actual_output=item['actual_output'],
            context=[item['expected_output']],
        )
        if item['cached_metrics_data'][0]['metric_metadata']['metric'] == 'Answer Relevancy':
            metric = AnswerRelevancyMetric(
                threshold=item['cached_metrics_data'][0]['metric_configuration']['threshold'],
                model = evaluateModel,
                include_reason=item['cached_metrics_data'][0]['metric_configuration']['include_reason']
            )
        elif item['cached_metrics_data'][0]['metric_metadata']['metric'] == 'Hallucination':
            metric = HallucinationMetric(
                threshold=item['cached_metrics_data'][0]['metric_configuration']['threshold'],
                model = evaluateModel,
                include_reason=item['cached_metrics_data'][0]['metric_configuration']['include_reason']
            )
        else:
            print("unkonwn metric!")
            continue
        
        try:
            metric.measure(test_case)
        except Exception as e:
            print(traceback.print_exc())
            errors_format_ins['id'] = item['id']
            errors_format_ins['AnswerModel'] = item['AnswerModel']
            errors_format_ins['input'] = item['input']
            errors_format_ins['actual_output'] = item['actual_output']
            errors_format_ins['expected_output'] = item['expected_output']
            errors_format_ins['cached_metrics_data'][0]['metric_metadata']['metric'] = metric.__name__
            errors_format_ins['cached_metrics_data'][0]['metric_configuration'] = {'threshold':metric.threshold,'evaluation_model':metric.evaluation_model,'strict_mode':metric.strict_mode,'include_reason':metric.include_reason}
            error['data'].append(errors_format_ins.copy())
            print(errors_format_ins)
            with open(errorFile,'w') as f:
                json.dump(data_er,f,indent=4)
            continue
        metrics_format_ins['metric_metadata']['metric'] = metric.__name__
        metrics_format_ins['metric_metadata']['score'] = metric.score
        metrics_format_ins['metric_metadata']['success'] = metric.is_successful()
        metrics_format_ins['metric_metadata']['reason'] = metric.reason
        
        metrics_format_ins['metric_metadata']['statements'] = getattr(metric,'statements','')
        metrics_format_ins['metric_metadata']['verdicts'] = str(getattr(metric,'verdicts',''))
        metrics_format_ins['metric_metadata']['evaluationCost'] = metric.evaluation_cost

        metrics_format_ins['metric_configuration']['threshold'] = metric.threshold
        metrics_format_ins['metric_configuration']['strict_mode'] = metric.strict_mode
        metrics_format_ins['metric_configuration']['evaluation_model'] = metric.evaluation_model
        metrics_format_ins['metric_configuration']['include_reason'] = metric.include_reason

        data_format_ins['cached_metrics_data'].append(metrics_format_ins.copy())
        flag = True
        for each in data_sv['data']:
            if is_same_eval_item(each, data_format_ins):
                each['cached_metrics_data'].append(metrics_format_ins.copy())
                flag = False
                break
        if flag:
            data_sv['data'].append(data_format_ins.copy())
            print(data_format_ins)
        with open(saveFile,'w') as f:
            json.dump(data_sv,f,indent=4)
    
    with open(errorFile,'w') as f:
        json.dump(error,f,indent=4)

In [88]:
with open('./eval/error/2-5_DocRED_sample.json') as f:
    tmp = json.load(f)
tmp

{'fileName': '2-5_DocRED_sample.json',
 'class': 'Knowledge Understanding',
 'data': [{'id': 1103,
   'AnswerModel': 'baichuan2-13b-base',
   'input': "Please follow the above demonstration, extract relations from the [Question text].\nNote the relation need to be in the predefined set of relations.\nThe output format required to is the same as the demonstration, format:(<entity_ID>, relation, <entity_ID>).\nThe predefined set of relations:\n{head of government, country, place of birth, place of death, father, mother, spouse, country of citizenship, continent, instance of, head of state, capital, official language, position held, child, author, member of sports team, director, screenwriter, educated at, composer, member of political party, employer, founded by, league, publisher, owned by, located in the administrative territorial entity, genre, operator, religion, contains administrative territorial entity, follows, followed by, headquarters location, cast member, producer, award rece

In [89]:
# with open('./eval/error/2-4_FewNERD++supervised_sample.json','r') as f:
#     data = json.load(f)
# for item in data['data']:
#     item['actual_output'] = item['actual_output'].replace("[/INST]",'')
#     item['actual_output'] = item['actual_output'].replace("[INST]",'')
# with open('./eval/error/2-4_FewNERD++supervised_sample.json','w') as f:
#     json.dump(data,f,indent=4)

In [90]:
files_error = ['1-1_2_high_freq_ent_sample.json',]# '1-2_1_low_freq_ent_sample.json', '1-3_r_1_simple_sample_sample.json', '2-2_COPEN++cpj_sample.json', '2-4_FewNERD++inter_sample.json', '2-4_FewNERD++intra_sample.json', '2-4_FewNERD++supervised_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json',  '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json',  '4-1_with_triples_sample.json', '4-2_r_with_triples_sample.json']
for file in files_error:
    joinEvalErrorToData('./eval/error/'+file,'./eval/save/'+file)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

In [ ]:

joinEvalErrorToData('./eval/error/2-5_DocRED_sample.json','./eval/save/2-5_DocRED_sample.json')

In [ ]:
path[0].text

In [ ]:
regex = re.compile("\"reason\":(.*?)\}",re.DOTALL)
matchStr = regex.findall(path[1])
# matchStr = regex.sub(lambda x:"\"reason\":\""+x.group(0).strip().replace("\"reason\":","").strip()[1:-1].replace('"','\\\"'),path[1])
matchStr

In [ ]:
matchStr[2].strip()[1:-1].replace('"','\\\"')

In [ ]:
regex.sub()

In [ ]:
path[1]

In [ ]:
['{"verdicts": [\n        {\n            "verdict": "idk"\n        },\n        {\n            "verdict": "no",\n            "reason": "The statement \'I am unable to provide an answer to your question\' acknowledges the inability to answer but doesn\'t provide any relevant information about the similarity with Rose."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'it is not clear what "Rose" refers to\' is relevant as it points out the need for clarification to determine similarity."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'Could you please provide more context or information?\' is relevant as it directly asks for additional details to establish the similarity with Rose."\n        }\n    ]}',
 '{"verdicts": [\n        {\n            "verdict": "no",\n            "reason": "The statement \'I\'m sorry,\' does not provide any information or similarity comparison with Rose."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'I need more context to answer your question.\' is relevant as it acknowledges the need for additional information to determine similarity with Rose."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'Could you please clarify what "Rose" refers to?\' is relevant as it seeks to understand the specific context or entity named Rose for a proper comparison."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'What you are looking for in terms of a conceptually similar entity?\' is relevant as it asks for the criteria or aspects to consider when identifying a conceptually similar entity to Rose."\n        }\n    ]}']

In [ ]:
def has_answer_kilt(answers, text) -> bool:
    text = normalize_kilt(text)
    for single_answer in answers:
        single_answer = normalize_kilt(single_answer)
        if single_answer in text:
            return True
    return False


# answer normalization
def normalize_kilt(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
# for file in fileNames:
#     evaluate_file('./data/'+file,'./eval/save/'+file,'./eval/error/'+file)

In [ ]:
with open('./eval/save/2-1_COPEN++csj_sample.json') as f:
    data_view = json.load(f)
noEvalItem = []
for item in data_view['data']:
    if len(item['cached_metrics_data']) <2:
        noEvalItem.append(item)
noEvalItem

In [ ]:
item = noEvalItem[0]

test_case = LLMTestCase(
    input= item['input'],
    actual_output=item['actual_output'],
    context=[item['expected_output']],
)
metric = AnswerRelevancyMetric(
    threshold=0.5,
    model = evaluateModel,
    include_reason=True,
)
metric.measure(test_case)


In [ ]:
for item in [noEvalItem[0]]:
    metrics_format_ins = {
        'metric_metadata':{
            'metric':None,
            'success':True,
            'score':0.8,
            'reason':'',
            'statements':'',
            'verdicts':'',
            'evaluationCost': 0
        },
        'metric_configuration': {
            'threshold': 0.5,
            'evaluation_model': 'CustomLLM',
            'strict_mode': False,
            'include_reason': True
        }
    }
    test_case = LLMTestCase(
        input= item['input'],
        actual_output=item['actual_output'],
        context=[item['expected_output']],
    )
    metric = AnswerRelevancyMetric(
        threshold=0.5,
        model = evaluateModel,
        include_reason=True,async_mode=False
    )
    metric.measure(test_case)
    
    metrics_format_ins['metric_metadata']['metric'] = metric.__name__
    metrics_format_ins['metric_metadata']['score'] = metric.score
    metrics_format_ins['metric_metadata']['success'] = metric.is_successful()
    metrics_format_ins['metric_metadata']['reason'] = metric.reason
    
    metrics_format_ins['metric_metadata']['statements'] = getattr(metric,'statements','')
    metrics_format_ins['metric_metadata']['verdicts'] = str(getattr(metric,'verdicts',''))
    metrics_format_ins['metric_metadata']['evaluationCost'] = metric.evaluation_cost

    metrics_format_ins['metric_configuration']['threshold'] = metric.threshold
    metrics_format_ins['metric_configuration']['strict_mode'] = metric.strict_mode
    metrics_format_ins['metric_configuration']['evaluation_model'] = metric.evaluation_model
    metrics_format_ins['metric_configuration']['include_reason'] = metric.include_reason
    
    print(metrics_format_ins)

### MATH PROCESS AND EVAL

In [ ]:
# with open('./data/math401/math50.json') as f:
#     data = json.load(f)
# data_new  = {'fileName': data['fileName'],'data':[]}
# for item in data['data']:
#     data_format = {'id':item['id'],'input':item['input'],'expected_output':item['expected_output']}
#     data_new['data'].append(data_format.copy())
# with open('./data/math401/math50.json','w') as f:
#     json.dump(data_new,f,indent=4)

In [ ]:
with open('./data/math401/math50.json','r') as f:
    data_eval  = json.load(f)
data_eval['data']

In [ ]:
# with open('./data/math401/save/infini-megrez-7b.json','r') as f:
#     data_401 = json.load(f)
#     data_401_new = {'fileName': data_401['fileName'],'data':[]}
#     for item in data_401['data']:
#         for x in data['data']:
#             if item['input'] =='Only return the correct answer of the question.\n'+ x['input']:
#                 item['id'] = x['id']
#                 item['is_correct'] = -1
#                 data_401_new['data'].append(item.copy())
# with open('./data/math401/save/infini-megrez-7b.json','w') as f:
#     json.dump(data_401_new,f,indent=4)

In [ ]:
def sort_result(filePath:Union[str,Path])->None:
    """
    _summary_
        the function is used to sort the results saved in the JSON `filepath` by `id` field
        
    Args:
        `filePath` (Union[str,Path]): the JSON file path

    Returns:
        None: the result will override the original `filePath` 
    """
    with open(filePath,'r') as f:
        data = json.load(f)
    data['data'].sort(key = lambda x:x['id'])
    with open(filePath,'w') as f:
        json.dump(data,f,indent=4)


In [ ]:
def addField(file:Union[str,Path],field:str,default:Any)->None:
    """
    _summary_
        the function is used to add a field into the `.json` `file`

    Args:
        `file` (Union[str,Path]): the filename ,the file should be JSON file
        `field` (str): the field name
        `default` (Any): the default value of the added field,the value will add or subtract a Random from -500_000_000 to 500_000_000 if the field is `time`
        
    Returns:
        None: the result will override the original `file` 
    """
    with open(file,'r') as f:
        data = json.load(f)
    for item in data['data']:
        if field not in item:
            item[field] = default
            if field == 'time':
                item[field] += random.randint(-500_000_000,500_000_000)
    with open(file,'w') as f:
        json.dump(data,f,indent = 4)

In [ ]:
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        addField(os.path.join(dir,file),'extract_answer',None)

In [ ]:
def getNumberAnswer(text:str)->Union[float,None]:
    """_summary_
        the function is used to extract the number from the text

    Args:
        `text` (str): the text contains the number
        
    Returns:
        Union[float,None]: the number in the `text`, or None if the `text` does not contain the number
    """
    text = text.split('=')
    if text:
        text = text[-1]
        regex = re.compile('([+-]?\d+[,0-9]*[.]?[0-9]*)')
        ret = regex.findall(text)
        if ret:
            return eval(ret[-1].replace(',',''))
        else:
            return None

In [ ]:
def processAnswer(file:Union[str,Path]):
    """
    _summary_:
        the function is used to process the answer that the LLM returned , namely extracting the number from the `actual_output` and save as `extract_answer`, and transform `expected_output` from str to number[int,float] 

    Args:
        `file` (Union[str,Path]): the filename of the file saving the answer created by the LLM
        
    Returns:
        None : the result will be written into the original `file`
    """
    with open(file,'r') as f:
        data = json.load(f)
    for item in data['data']:
        if item['is_correct'] == -1:
            item['extract_answer'] = getNumberAnswer(item['actual_output'])
            item['expected_output'] = eval(item['expected_output'])
            if item['extract_answer'] is None :
                item['is_correct'] = 0
            else:
                item['is_correct'] =1 if abs(item['expected_output']-item['extract_answer']) < 1e-3 else 0
    with open(file,'w') as f:
        json.dump(data,f,indent = 4)

In [ ]:
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        processAnswer(os.path.join(dir,file))

In [ ]:
# for dir,subdir ,files in os.walk('./data/math401/save'):
#     for file in files:
#         with open(os.path.join(dir,file),'r') as f:
#             data = json.load(f)
#         for item in data['data']:

#             if item['is_correct']:
#                 item['is_correct'] = 1
#             else:
#                 item['is_correct'] = 0
#         with open(os.path.join(dir,file),'w') as f:
#             json.dump(data,f,indent = 4)

In [ ]:

def calculate_accuracy(file:Union[str,Path],return_tuple:bool = False)->Union[float,tuple[str,float]]:
    """_summary_
        the function calculates the accuracy of the model answers on the `math50` dataset saved in `file`

    Args:
        `file` (Union[str,Path]): the path to the JSON `file` containing a model answer results
        `return_tuple` (bool, optional): if the value is True,the function will return a tuple (modelName,accuracy),else only return accuracy `[0,1]`. 
                    Defaults to False.

    Returns:
        Union[float,tuple[str,float]]: if return_tuple is True,return a tuple containing both modelName and accuracy in format `(modelName:str,accuracy:float)`,else only return the accuracy in [0,1]
    """
    with open(file,'r') as f:
        data = json.load(f)
    correct = 0
    for item in data['data']:
        correct += item['is_correct']
    accuracy =  correct/len(data['data'])
    if return_tuple:
        return (data['model'],accuracy)
    return accuracy

def calculate_nan_ratio(file:Union[str,Path],return_tuple:bool = False)->Union[float,tuple[str,float]]:
    """_summary_
        the function calculates the no number ratio of the LLM answers on the `math50` dataset saved in `file`

    Args:
        `file` (Union[str,Path]): the path to the JSON `file` containing a model answer results
        `return_tuple` (bool, optional): if the value is True,the function will return a tuple (modelName,nan_ratio),else only return nan_ratio `[0,1]`. Defaults to False.

    Returns:
        Union[float,tuple[str,float]]: if return_tuple is True,return a tuple containing both modelName and nan_ratio in format `(modelName:str,nan_ratio:float)`,else only return the nan_ratio in [0,1]
    """
    with open(file,'r') as f:
        data = json.load(f)
    nan = 0
    for item in data['data']:
        if item['extract_answer'] is None:
            nan += 1
    nan_ratio = nan/len(data['data'])
    if return_tuple:
        return (data['model'],nan_ratio)
    return nan_ratio

In [ ]:
def RE(y_true,y_pred):
    return min(10,abs(y_true-y_pred)/max(abs(y_true),1))
def calculate_RE(file:Union[str,Path],return_tuple:bool = False)->Union[List[float],tuple[str,List[float]]]:
    with open(file,'r') as f:
        data = json.load(f)
    RE_list = []
    for item in data['data']:
        ret = 0
        y_true = item['expected_output']
        y_pred = item['extract_answer']
        if y_pred is None:
            ret = 10
        else:
            ret = RE(y_true,y_pred)
        RE_list.append(ret)
    if return_tuple:
        return (data['model'],RE_list)
    return RE_list

In [ ]:
# data_list = {'eval':'math','eval_dataset':'math50.json','data':[]}
# for dir,subdir ,files in os.walk('./data/math401/save'):
#     for file in files:
#         tmp = calculate_accuracy(os.path.join(dir,file),return_tuple=True)
#         nan_ratio = calculate_nan_ratio(os.path.join(dir,file))
#         RE_List = calculate_RE(os.path.join(dir,file))
#         data_list['data'].append({'model':tmp[0],'Accuracy':tmp[1],'Nan_Ratio':nan_ratio,'RE_List':RE_List[:]})
# with open('./data/math401/math50_eval_result.json','w') as f:
#     json.dump(data_list,f,indent=4)

In [2]:
from evalplus.data import get_human_eval_plus

In [3]:
dataset = get_human_eval_plus()

In [5]:
len(dataset)

164

In [2]:
os.environ.get("HUMANEVAL_OVERRIDE_PATH", None)

In [2]:
with open('./data/codeEval/data.json','r') as f:
    data_eval = json.load(f)


In [32]:
data_eval['HumanEval/12']

{'task_id': 'HumanEval/12',
 'prompt': 'from typing import List, Optional\n\n\ndef longest(strings: List[str]) -> Optional[str]:\n    """ Out of list of strings, return the longest one. Return the first one in case of multiple\n    strings of the same length. Return None in case the input list is empty.\n    >>> longest([])\n\n    >>> longest([\'a\', \'b\', \'c\'])\n    \'a\'\n    >>> longest([\'a\', \'bb\', \'ccc\'])\n    \'ccc\'\n    """\n',
 'entry_point': 'longest',
 'canonical_solution': '\n    if not strings:\n        return None\n\n    maxlen = max(len(x) for x in strings)\n    for s in strings:\n        if len(s) == maxlen:\n            return s\n\n',
 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([]) == None\n    assert candidate(['x', 'y', 'z']) == 'x'\n    assert candidate(['x', 'yyy', 'zzzz', 'www', 'kkkk', 'abc']) == 'zzzz'\n",
 'contract': '\n    assert type(strings) == list, "invalid inputs" # $_

In [ ]:
# with open('./data/codeEval/data.jsonl','w') as fw:
#     for item in data.keys():
#         fw.write(json.dumps(data[item])+'\n')

In [ ]:
# data = {}
# for dir,subdir ,files in os.walk('./data/math401/save'):
#     for file in files:
#         tmp = calculate_accuracy(os.path.join(dir,file),return_tuple=True)
#         nan_ratio = calculate_nan_ratio(os.path.join(dir,file))
#         RE_List = calculate_RE(os.path.join(dir,file))
#         data[tmp[0]] = {'Accuracy':tmp[1],'Nan_Ratio':nan_ratio,'RE_List':RE_List[:]}
# df = pd.DataFrame(data).T.sort_index()
# df.to_json('./data/math401/math50_eval_result_df.json')

In [55]:
data_eval = []
with open('./data/codeEval/code_sanitize/qwen-72b.jsonl') as f:
    for line in f.readlines():
        data_eval.append(json.loads(line))


In [56]:
data_eval[1]['solution']

"from typing import List\ndef parse_nested_parens(paren_string: str) -> List[int]:\n    result = []\n    for group in paren_string.split():\n        stack = []\n        max_depth = 0\n        for char in group:\n            if char == '(':\n                stack.append(char)\n                if len(stack) > max_depth:\n                    max_depth = len(stack)\n            elif char == ')':\n                stack.pop()\n        result.append(max_depth)\n    return result"

In [57]:
exec_globals = {}
exec(data_eval[1]['solution'],exec_globals)

'HumanEval/6'

In [62]:
fn = exec_globals['parse_nested_parens']

In [59]:
with open('./data/codeEval/data.json') as f:
    data_o = json.load(f)

In [61]:
data_o['HumanEval/6']

{'task_id': 'HumanEval/6',
 'prompt': 'from typing import List\n\n\ndef parse_nested_parens(paren_string: str) -> List[int]:\n    """ Input to this function is a string represented multiple groups for nested parentheses separated by spaces.\n    For each of the group, output the deepest level of nesting of parentheses.\n    E.g. (()()) has maximum two levels of nesting while ((())) has three.\n\n    >>> parse_nested_parens(\'(()()) ((())) () ((())()())\')\n    [2, 3, 1, 3]\n    """\n',
 'entry_point': 'parse_nested_parens',
 'canonical_solution': '\n    \n    def count_depth(s: str) -> int:\n        max_depth, cnt = 0, 0\n        for ch in s:\n            if ch == "(": cnt += 1\n            if ch == ")": cnt -= 1\n            max_depth = max(max_depth, cnt)\n        return max_depth\n    \n    return [count_depth(s) for s in paren_string.split(" ") if s != ""]\n\n',
 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidat

In [16]:
env = {}

In [18]:
env

{'__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'),
  '__build_class__': <function __build_class__>,
  '__import__': <function __import__>,
  'abs': <function abs(x, /)>,
  'all': <function all(iterable, /)>,
  'any': <function any(iterable, /)>,
  'ascii': <function ascii(obj, /)>,
  'bin': <function bin(number, /)>,
  'breakpoint': <function breakpoint>,
  'callable': <function callable(obj, /)>,
  'chr': <function chr(i, /)>,
  'compile': <function compile(source, filename, mode, flags=0, dont_inherit=False, optimize=-1, *, _feature_version=-1)>,
  'delattr': <function delattr(obj, name, /)>,
  'dir': <function dir>,
  'divmod': <function divmod(x, y, /)>,
  'eval': <

In [17]:
exec("",env)

In [64]:
exec(data_o['HumanEval/6']['prompt']+data_o['HumanEval/6']['canonical_solution'],env)

In [66]:
fn_st = env[data_o['HumanEval/6']['entry_point']]

In [73]:
ret = []
ret_st = []
for inp in  data_o['HumanEval/6']['plus_input']:
    inp = inp[:]
    tmp = inp[:]
    ret.append(fn(*inp))
    ret_st.append(fn_st(*tmp))

In [74]:
data_o['HumanEval/6']['plus_input']

[[''],
 ['((()))'],
 ['(())(()())'],
 ['(())(()(()))((()()))'],
 ['(()()(((())))(()(())))()'],
 ['()((()))'],
 ['(())'],
 ['()()()'],
 ['()(())'],
 ['(((())(()(()))((()()))))(()(()))((()()))'],
 ['(()()(()(())((()()(((())))(()(())))())(((()))))(()(())))()'],
 ['((()(())(()(()))((()()))))'],
 ['(()()(((())))(()(())(((())(()(()))((()()))))(()(()))((()()))))()'],
 ['((()()(((())))(()(())))())((()()))'],
 ['(()()(((())((()()(((())))(()(())))())(((()))))(()(()))))()'],
 ['()(()()(((())))(()(())(((())(()(()))((()()))))(()(()))((()()))))()((()))'],
 ['((()(())((()(()))((()())))))'],
 ['((((((()()()((()))(())()((((()))))))))))((((((()()()((()))(())()((((()))))))))))((((((()()()((()))(())()((((()))))))))))((((((()()()((()))(())()((((()))))))))))((((((()()()((()))(())()((((()))))))))))((((((()()()((()))(())()((((()))))))))))((((((()()()((()))(())()((((()))))))))))'],
 ['((((((())(())))())((()()()(()(((()(()))))))))()(((())(()(()))((()()()))()))((()))()(()))()'],
 ['()()()()()()'],
 ['((((((()))(

In [75]:
n = len(ret)
acc = 0
for i in range(n):
    acc += 1 if ret[i] == ret_st[i] else 0
acc/n

1.0

In [76]:
ret_st

[[],
 [3],
 [2],
 [3],
 [5],
 [3],
 [2],
 [1],
 [2],
 [5],
 [8],
 [5],
 [7],
 [6],
 [9],
 [7],
 [6],
 [11],
 [10],
 [1],
 [8],
 [4],
 [3],
 [1],
 [5],
 [6],
 [10],
 [5],
 [4],
 [8],
 [9],
 [3],
 [6],
 [3],
 [5],
 [3],
 [5],
 [16],
 [13],
 [11],
 [6],
 [11],
 [3],
 [1],
 [22],
 [10],
 [15],
 [17],
 [19],
 [6],
 [3],
 [11],
 [11],
 [22],
 [12],
 [12],
 [8],
 [8],
 [1],
 [14],
 [14],
 [16],
 [10],
 [11],
 [18],
 [20]]

In [71]:
ret

[[],
 [3],
 [2],
 [3],
 [5],
 [3],
 [2],
 [1],
 [2],
 [5],
 [8],
 [5],
 [7],
 [6],
 [9],
 [7],
 [6],
 [11],
 [10],
 [1],
 [8],
 [4],
 [3],
 [1],
 [5],
 [6],
 [10],
 [5],
 [4],
 [8],
 [9],
 [3],
 [6],
 [3],
 [5],
 [3],
 [5],
 [16],
 [13],
 [11],
 [6],
 [11],
 [3],
 [1],
 [22],
 [10],
 [15],
 [17],
 [19],
 [6],
 [3],
 [11],
 [11],
 [22],
 [12],
 [12],
 [8],
 [8],
 [1],
 [14],
 [14],
 [16],
 [10],
 [11],
 [18],
 [20]]

In [1]:
from evalplus.evaluate import main

In [2]:
from sys import argv

In [3]:
argv.pop()

'--f=c:\\Users\\游锦亮\\AppData\\Roaming\\jupyter\\runtime\\kernel-v2-19400V4QYjsjrB1Su.json'

In [4]:
local_args = argv
argv

['d:\\Anaconda\\envs\\vllm\\lib\\site-packages\\ipykernel_launcher.py']

In [5]:
local_args.append('--dataset=humaneval')
local_args.append('--samples=./data/codeEval/code_sanitize/qwen-72b.jsonl')
local_args.append('--test-details')

In [6]:
main()

Load from ground-truth from C:\Users\游锦亮\AppData\Local\evalplus\evalplus\Cache\1376469eb11e2af066a52cb042f854bf.pkl
Reading samples...


28it [00:00, 569.87it/s]
100%|██████████| 28/28 [00:03<00:00,  8.36it/s]


humaneval (base tests)
pass@1:	0.000
humaneval+ (base + extra tests)
pass@1:	0.000


In [44]:
with open('data/codeEval/data.json', 'r') as f:
    data_eval = json.load(f)

In [43]:
from evalplus.gen.util import trusted_exec
from evalplus.eval import (
    FAIL,
    PASS,
    compatible_eval_result,
    estimate_pass_at_k,
    untrusted_check,
)
from evalplus.gen.util import trusted_exec

In [52]:
def get_groundtruth(problems, tasks_only_output_not_none):
    print("Computing expected output...")
    tbegin = time.time()
    expected_output = {}
    for task_id, problem in problems.items():
        oracle = {}
        oracle["base"], oracle["base_time"] = trusted_exec(
            problem["prompt"] + problem["canonical_solution"],
            problem["base_input"],
            problem["entry_point"],
            record_time=True,
            output_not_none=problem["entry_point"] in tasks_only_output_not_none,
        )

        oracle["plus"], oracle["plus_time"] = trusted_exec(
            problem["prompt"] + problem["canonical_solution"],
            problem["plus_input"],
            problem["entry_point"],
            record_time=True,
            output_not_none=problem["entry_point"] in tasks_only_output_not_none,
        )
        expected_output[task_id] = oracle
    print(f"Expected outputs computed in {time.time() - tbegin:.2f}s")

    with open('./test.json', "w") as f:
        json.dump(expected_output, f,indent=4)

    return expected_output

In [ ]:
def is_floats(x) -> bool:
    # check if it is float; List[float]; Tuple[float]
    if isinstance(x, float):
        return True
    if isinstance(x, (list, tuple)):
        return all(isinstance(i, float) for i in x)
    if isinstance(x, np.ndarray):
        return x.dtype == np.float64 or x.dtype == np.float32
    return False


def unsafe_execute(
    dataset: str,
    entry_point: str,
    code: str,
    inputs,
    expected: List,
    time_limits,
    atol,
    fast_check,
    stat: Value,
    details: Array,
    progress: Value,
):
    with create_tempdir():
        # These system calls are needed when cleaning up tempdir.
        import os
        import shutil

        rmtree = shutil.rmtree
        rmdir = os.rmdir
        chdir = os.chdir
        # Disable functionalities that can make destructive changes to the test.
        # allow only 4GB memory usage
        maximum_memory_bytes = 4 * 1024 * 1024 * 1024
        reliability_guard(maximum_memory_bytes=maximum_memory_bytes)
        exec_globals = {}
        try:
            with swallow_io():
                exec(code, exec_globals)
                fn = exec_globals[entry_point]
                for i, inp in enumerate(inputs):
                    try:
                        with time_limit(time_limits[i]):
                            out = fn(*inp)

                        exp = expected[i]
                        exact_match = out == exp

                        # ================================================ #
                        # ============== special oracles ================= #
                        if dataset == "mbpp":
                            if (
                                "are_equivalent" == entry_point
                            ):  # Mbpp/164 special oracle
                                exact_match = exact_match or True
                            elif "sum_div" == entry_point:  # Mbpp/295 special oracle
                                exact_match = exact_match or out == 0
                            elif entry_point in MBPP_OUTPUT_NOT_NONE_TASKS:
                                # exp is True  if not None
                                #        False if None
                                if isinstance(out, bool):
                                    exact_match = out == exp
                                else:
                                    exact_match = exp == (out is not None)

                        if dataset == "humaneval":
                            if "find_zero" == entry_point:
                                assert _poly(*inp, out) <= atol
                        # ============== special oracles ================= #
                        # ================================================ #

                        if atol == 0 and is_floats(exp):
                            atol = 1e-6  # enforce atol for float comparison
                        if not exact_match and atol != 0:
                            np.testing.assert_allclose(out, exp, atol=atol)
                        else:
                            assert exact_match
                    except BaseException:
                        if fast_check:
                            raise

                        details[i] = False
                        progress.value += 1
                        continue

                    details[i] = True
                    progress.value += 1
            stat.value = _SUCCESS
        except BaseException:
            stat.value = _FAILED
        # Needed for cleaning up.
        shutil.rmtree = rmtree
        os.rmdir = rmdir
        os.chdir = chdir


In [2]:
import eventlet
import time
from func_timeout import *

In [3]:
path = []

In [4]:
def execute(code:str,inputs:List,entry_point:str,record_time=True)->List:
    env = {}
    n = len(inputs)
    try:
        exec(code,None, env)
    except:
        if record_time:
            return (["SyntaxError"]*n,[(1<<31)-1]*n)
        return ["SyntaxError"]*n
    if entry_point in env:
        fn = env[entry_point]
    else:
        if record_time:
            return (["NotImplemented"]*n,[(1<<31)-1]*n)
        return ["NotImplemented"]*n
    @func_set_timeout(1)
    def get(inp,record_time:bool=True):
        try:
            start = time.perf_counter_ns()
            ret = fn(*inp)
            end = time.perf_counter_ns()
            delta = (end-start)//1000_000
        except:
            ret = "SyntaxError"
            delta = (1<<31)-1
        if isinstance(ret,NotImplemented) or isinstance(ret,NotImplementedError) or str(ret) == str(NotImplemented):
            ret = "NotImplemented"
        if record_time:
            return (ret,delta)
        return ret
        
    ret = []
    rtime = []
    for i,inp in enumerate(inputs):
        try:
            res,delta = get(inp)
            ret.append(res)
            rtime.append(delta)
        except:
            ret.append("TimeLimitExceeding")
            rtime.append((1<<31)-1)
    del env[entry_point]
    if record_time:
        return ret,rtime
    return ret

In [5]:
def codeExecute(file_path:Union[str,Path],dataset:Union[str,Path],exec_save_path:Union[str,Path] = None,is_force_override = False)->None:
    
    if isinstance(dataset, str):
        dataset = Path(dataset)
    if dataset.suffix == ".json":
        with open(dataset,'r') as f:
            data_origin = json.load(f)
    elif dataset.suffix == '.jsonl':
        with open(dataset,'r') as f:
            # {"task_id":{
            #     "task_id":"",
            #     "prompt":"",
            #     "entry_point":"",
            #     "canonical_solution":"",
            #     "base_input":"",
            #     "plus_input":""
            #     }
            # }
            data_origin = {}
            for line in f:
                item = json.loads(line)
                data_origin[item['task_id']] = item
    else:
        raise Exception(f"{dataset.suffix} is not supported")
    
    if isinstance(file_path, str):
        file_path = Path(file_path)
    if file_path.suffix == ".json":
        with open(file_path,'r') as f:
            data_eval = json.load(f)
    elif file_path.suffix == '.jsonl':
        with open(file_path,'r') as f:
            # {"task_id":"solution",}
            data_eval = {}
            for line in f:
                item = json.loads(line)
                task_id = item['task_id']
                data_eval[task_id] = item['solution'] if 'solution' in item else data_origin[task_id]['prompt']+item['completion']
    else:
        raise Exception(f"{file_path.suffix} is not supported")
    """
        {
            "{{task_id}}":{
                "info":{
                    "task_id":"{{task_id}}",
                    "prompt":"{{prompt}}",
                    "entry_point":"{{entry_point}}",
                    "canonical_solution":"{{canonical_solution}}",
                    "base_input":"{{base_input}}",
                    "plus_input":"{{plus_input}}",
                },
                "expected_output":{
                    "base":"{{base}}",
                    "base_time":"{{base_time}}",
                    "plus":"{{plus}}",
                    "plus_time":"{{plus_time}}",
                },
                "code_LLM":"{{solution}}",
                "actual_output":{
                    "base":"{{base}}",
                    "base_time":"{{base_time}}",
                    "plus":"{{plus}}",
                    "plus_time":"{{plus_time}}",
                }
            }
        }
    """
    output = {}
    save = file_path.stem + "_execute_results.json"
    if not exec_save_path:
        parent = file_path.parent
    else:
        if isinstance(exec_save_path, str):
            exec_save_path = Path(exec_save_path)
        parent = exec_save_path if os.path.isdir(exec_save_path) else exec_save_path.parent
    if os.path.exists(os.path.join(parent,save)) and not is_force_override:
        print("The executeResults file already exists")
        return
    for task_id,problem in data_origin.items():
        oracle = {}
        oracle["base"], oracle["base_time"] = execute(
            problem["prompt"] + problem["canonical_solution"],
            problem["base_input"],
            problem["entry_point"],
            record_time=True,
        )
        oracle["plus"], oracle["plus_time"] = execute(
            problem["prompt"] + problem["canonical_solution"],
            problem["plus_input"],
            problem["entry_point"],
            record_time=True,
        )
        if task_id not in output:
            output[task_id] = {}
        output[task_id]['info'] = problem
        output[task_id]["expected_output"] = oracle
    for task_id,solution in data_eval.items():
        oracle = {}
        oracle["base"], oracle["base_time"] = execute(
            solution,
            data_origin[task_id]["base_input"],
            data_origin[task_id]["entry_point"],
            record_time=True,
        )
        oracle["plus"], oracle["plus_time"] = execute(
            solution,
            data_origin[task_id]["plus_input"],
            data_origin[task_id]["entry_point"],
            record_time=True,
        )
        if task_id not in output:
            output[task_id] = {}
        output[task_id]['code_LLM'] = solution
        output[task_id]["actual_output"] = oracle
    path.append(output)
    with open(os.path.join(parent,save), "w") as f:
        json.dump(output, f, indent=4,)
    print(save)
def is_floats(x) -> bool:
    # check if it is float; List[float]; Tuple[float]
    if isinstance(x, float):
        return True
    if isinstance(x, (list, tuple)):
        return all(isinstance(i, float) for i in x)
    if isinstance(x, np.ndarray):
        return x.dtype == np.float64 or x.dtype == np.float32
    return False

In [6]:
def codeEvaluate(exec_file:Union[str,Path],eval_save_path:Union[str,Path] = None,is_force_override:bool = False):
    def equal(exp:List,actual:List)->List[bool]:
        ret_correct = []
        if is_floats(exp):
            for i in range(len(exp)):
                if isinstance(actual[i],str) or not np.isclose(exp[i],actual[i],atol=1e-5):
                    ret_correct.append(False)
                else:
                    ret_correct.append(True)
        else:
            for i in range(len(exp)):
                if actual in ["NotImplemented","TimeLimitExceeding","SyntaxError"] or actual[i]!= exp[i]:
                    ret_correct.append(False)
                else:
                    ret_correct.append(True)
        return ret_correct
    if isinstance(exec_file, str):
        exec_file = Path(exec_file)
    if exec_file.suffix != ".json":
        raise Exception(f"{exec_file.suffix} is not supported")
    with open(exec_file,'r') as f:
        data = json.load(f)
    if not eval_save_path:
        parent = exec_file.parent
    else:
        if isinstance(eval_save_path,str):
            eval_save_path = Path(eval_save_path)
        parent = eval_save_path if os.path.isdir(eval_save_path) else eval_save_path.parent
            
    save = exec_file.stem.replace('_execute_results','') + "_eval_results.json"
    if os.path.exists(os.path.join(parent,save)) and not is_force_override:
        print("The evalResults file already exists")
        return 
    eval_result = {}
    """
        {
            "percision":{
                "base":float,
                "plus":float
            },
            "accuracy_list":{
                "base_accuracy_list":List[float],
                "plus_accuracy_list":List[float]
            },
            "syntax_error_ratio":{
                "base":float,
                "plus":float
            },
            "syntax_error_list":{
                "base_syntax_error_list":List[bool],
                "plus_syntax_error_list":List[bool]
            },
            "notImplemented_ratio_list":List[float],
            "{{task_id}}":{
                "base_eval":List[bool],
                "base_num":int,
                "base_AC":int,
                "base_TLE":int,
                "base_accuracy":float,
                "base_exp_time":List[time],
                "base_actual_time":List[time],
                
                "plus_eval":List[bool],
                "plus_num":int,
                "plus_AC":int,
                "plus_TLE":int,
                "plus_accuracy":float,
                "plus_exp_time":List[time],
                "plus_actual_time":List[bool],
                
                "accuracy":float
            }
        }
    """
    eval_result['percision'] = {'base':0,'plus':0,"mean":0}
    eval_result['syntax_error_ratio'] = {'base':0,"plus":0,"mean":0}
    base_accuracy_list = []
    plus_accuracy_list = []
    accuracy_list = {"base_accuracy_list":base_accuracy_list,"plus_accuracy_list":plus_accuracy_list}
    eval_result['accuracy_list'] = accuracy_list
    base_syntax_error_list = []
    plus_syntax_error_list = []
    eval_result['syntax_error_list'] ={ "base_syntax_error_list":base_syntax_error_list,"plus_syntax_error_list":plus_syntax_error_list}
    notImplemented_list = []
    eval_result['notImplemented_ratio_list'] = notImplemented_list
    
    for task_id,eval in data.items():
        oracle = {}
        base_exp = eval['expected_output']['base']
        base_actual = eval['actual_output']['base']
        base_correct = equal(base_exp, base_actual)
        
        oracle['base_num'] = len(base_correct)
        oracle['base_AC'] = sum(base_correct)
        oracle['base_TLE'] = base_actual.count("TimeLimitExceeding")
        oracle['base_SE'] = base_actual.count("SyntaxError")
        oracle['base_NoImplemented'] = base_actual.count("NotImplemented")
        oracle['base_accuracy'] = np.mean(base_correct)
        base_accuracy_list.append(oracle['base_accuracy'])
        base_syntax_error_list.append(oracle['base_SE']/oracle['base_num'])
        
        plus_exp = eval['expected_output']['plus']
        plus_actual = eval['actual_output']['plus']
        plus_correct = equal(plus_exp,plus_actual)
        
        oracle['plus_num'] = len(plus_correct)
        oracle['plus_AC'] = sum(plus_correct)
        oracle['plus_TLE'] = plus_actual.count("TimeLimitExceeding")
        oracle['plus_SE'] = plus_actual.count("SyntaxError")
        oracle['plus_NoImplemented'] = plus_actual.count("NotImplemented")
        oracle['plus_accuracy'] = np.mean(plus_correct)
        plus_accuracy_list.append(oracle['plus_accuracy'])
        plus_syntax_error_list.append(oracle['plus_SE']/oracle['plus_num'])
        
        oracle['accuracy'] = np.mean(base_correct+plus_correct)
        
        
        oracle['base_exp_time'] = eval['expected_output']['base_time']
        oracle['base_actual_time'] = eval['actual_output']['base_time']
        oracle['plus_exp_time'] = eval['expected_output']['plus_time']
        oracle['plus_actual_time'] = eval['actual_output']['plus_time']
        
        notImplemented_list.append((oracle['base_NoImplemented']+oracle['plus_NoImplemented'])/(oracle['base_num']+oracle['plus_num']))
        
        eval_result[task_id] = oracle
    eval_result['percision']['base'] = np.mean(base_accuracy_list)
    eval_result['percision']['plus'] = np.mean(plus_accuracy_list)
    eval_result['percision']['mean'] = np.mean(base_accuracy_list+plus_accuracy_list)
    eval_result['syntax_error_ratio']['base'] = np.mean(base_syntax_error_list)
    eval_result['syntax_error_ratio']['plus'] = np.mean(plus_syntax_error_list)
    eval_result['syntax_error_ratio']['mean'] = np.mean(base_syntax_error_list+plus_syntax_error_list)
    
    with open(os.path.join(parent,save), "w") as f:
        json.dump(eval_result, f, indent=4,)
    print(save)

In [7]:
for dir,subdir,files in os.walk('./data/codeEval/code_sanitize'):
    for file in files:
        if file.endswith('.jsonl'):
            codeExecute(os.path.join(dir,file),"./data/codeEval/data.json",exec_save_path = './data/codeEval/code_execute',is_force_override=True)
            codeEvaluate(os.path.join('./data/codeEval/code_execute',file.replace(".jsonl","_execute_results.json")),eval_save_path = './data/codeEval/code_eval',is_force_override=True)

baichuan2-13b-base_execute_results.json
baichuan2-13b-base_eval_results.json
False
True
False
False
True
False
baichuan2-13b-chat_execute_results.json
baichuan2-13b-chat_eval_results.json
baichuan2-7b-chat_execute_results.json
baichuan2-7b-chat_eval_results.json
infini-megrez-7b_execute_results.json
infini-megrez-7b_eval_results.json
llama-2-13b-chat_execute_results.json
llama-2-13b-chat_eval_results.json
llama-2-70b_execute_results.json
llama-2-70b_eval_results.json
llama-2-7b-chat_execute_results.json
llama-2-7b-chat_eval_results.json
qwen-14b-chat_execute_results.json
qwen-14b-chat_eval_results.json
qwen-72b-chat_execute_results.json
qwen-72b-chat_eval_results.json
qwen-72b_execute_results.json
qwen-72b_eval_results.json
qwen-7b-chat_execute_results.json
qwen-7b-chat_eval_results.json
qwen1.5-14b-chat_execute_results.json
qwen1.5-14b-chat_eval_results.json
qwen1.5-72b_execute_results.json
qwen1.5-72b_eval_results.json
qwen1.5-7b-chat_execute_results.json
qwen1.5-7b-chat_eval_results

In [20]:
path[0]['HumanEval/54']['actual_output']['plus'][9].__class__

NotImplementedType

In [ ]:
path[0]['HumanEval/54']['actual_output']['plus'][9].__class__

NotImplementedType

In [2]:
[1,1,2,1].count(1)

3

In [8]:
env = {}
exec("def sum_to_n(n):    return sum(range(1,n+1))",None,env)

In [9]:
env

{'sum_to_n': <function __main__.sum_to_n(n)>}